## Import and configure

In [ ]:
# Imports from standard library
import os.path
import datetime
import warnings
warnings.filterwarnings('ignore')

# Import of third party packages
import numpy as np
import netCDF4
%matplotlib widget
from matplotlib import pyplot
from matplotlib import dates

import eatpy

# Configuration
setup_dir = "."           # directory with GOTM model setup
ensemble_dir = "."        # directory with GOTM model setup
N = 5                     # ensemble size
max_temp_diff = 2.0       # maximum difference (°C) to show in contour plot
cmap = 'viridis'          # color map to use for filled contour plots [alternative: cmocean.cm.thermal]

## Load observations

In [ ]:
with open(os.path.join(setup_dir, 'cci_sst.dat')) as f:
    obs = [line.rstrip().split('\t') for line in f if not line.startswith('#')]
obs_times = dates.date2num([datetime.datetime.strptime(o[0], '%Y-%m-%d %H:%M:%S') for o in obs])
obs_values = np.array([o[1] for o in obs], dtype=float)

# Chlorophyll
chl_obs = []
with open(os.path.join(setup_dir, 'cci_chl.dat')) as f:
    for l in f:
        if not l.startswith('#'):
            dt, value, sd = l.rstrip().split('\t')
            mu = float(value)
            sigma = float(sd)
            chl_obs.append([datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'), 10.**mu, 10.**(mu-0.67448*sigma), 10.**(mu+0.67448*sigma)])
chl_obs = np.array(chl_obs).reshape(-1, 4)

# Reference simulation

Forecast-only, no data assimilation

In [ ]:
!eat-gotm

## Load results

In [ ]:
# Load simulation results
with netCDF4.Dataset(os.path.join(setup_dir, 'result.nc')) as nc:
    time = netCDF4.num2date(nc['time'], nc['time'].units)
    mpltime = dates.date2num(time)
    z = -nc['z'][:, :, 0, 0]
    temp = nc["temp"][:, :, 0, 0]
    sst = temp[:, -1]
    chl = nc["total_chlorophyll_calculator_result"][:, :, 0, 0]
    chl_sf = chl[:, -1]

## Plot temperature

In [ ]:
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

# Plot modelled and observed sea surface temperature
ax1.plot_date(obs_times, obs_values, '.k', alpha=0.4, label='satellite')
ax1.plot_date(mpltime, sst, '-', label='model')
ax1.set_xlim(mpltime[0], mpltime[-1])
ax1.set_ylabel('temperature (°C)')
ax1.grid()
ax1.legend()
ax1.set_title('sea surface temperature')
cax1.axis('off')

# Plot modelled temperature throughout the water column
mpltime_2d = np.broadcast_to(mpltime[:, np.newaxis], z.shape)
pc = ax2.contourf(mpltime_2d, z, temp, 10, cmap=cmap)
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_ylabel('depth (m)')
ax2.set_title('model temperature')
ax2.grid()
ax2.xaxis.axis_date()
ax2.set_ylim(z.max(), z.min())

fig.tight_layout()

fig.savefig('reference_sst.png', dpi=150)

## Plot chlorophyll

In [ ]:
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

ax1.plot_date(chl_obs[:,0], chl_obs[:,1], '.k', label='satellite')
ax1.plot_date(mpltime, chl_sf, '-', color='C0', label='model')
ax1.set_xlim(mpltime[0], mpltime[-1])
ax1.set_ylabel('chlorophyll (mg m-3)')
ax1.grid()
ax1.legend()
ax1.set_title('surface chlorophyll')
cax1.axis('off')

mpltime_2d = np.broadcast_to(mpltime[:, np.newaxis], z.shape)
pc = ax2.contourf(mpltime_2d, z, chl, 20)
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('chlorophyll (mg m-3)')
ax2.set_ylabel('depth (m)')
ax2.set_title('chlorophyll')
ax2.xaxis.axis_date()
ax2.set_ylim(z.max(), z.min())
ax2.grid()

fig.tight_layout()
fig.savefig('reference_sst_chl.png', dpi=150)

# Physics-only data assimilation

## Create the ensemble

In [ ]:
# Vary wind speeds (x and y components) and background mixing (minimum turbulent kinetic energy) 
gotm = eatpy.models.gotm.YAMLEnsemble("gotm.yaml", N)
with gotm:
    gotm["surface/u10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["surface/v10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["turbulence/turb_param/k_min"] *= np.random.lognormal(sigma=0.2, size=N)

## Run data assimilation experiment

In [ ]:
!mpiexec -n 1 python assimilate_sst.py : -n {N} eat-gotm --separate_gotm_yaml

## Load results

In [ ]:
# Open NetCDF output of all ensemble members\n",
ncs = [netCDF4.Dataset(os.path.join(ensemble_dir, 'result_%04i.nc' % (i + 1))) for i in range(N)]

# Read model temperature and coordinates\n",
enstime = netCDF4.num2date(ncs[0]['time'], ncs[0]['time'].units)
ensmpltime = dates.date2num(enstime)
z = -ncs[0].variables['z'][:, :, 0, 0]
temps = [nc[varname][:, :, 0, 0] for nc in ncs]
ssts = [temp[:, -1] for temp in temps]

## Plot

In [ ]:
# Plot ensemble mean sea surface temperature, along with original (no DA) result and observations
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(figsize=(8,8), nrows=3, ncols=2, width_ratios=[0.95, 0.05], sharex='col')
ax1.plot_date(obs_times, obs_values, '.k', alpha=0.4, label='satellite')
ax1.plot_date(time, sst, '-', label='model, no DA')
ax1.plot_date(enstime, np.mean(ssts, axis=0), '-', label='model, DA')
ax1.set_xlim(time[0], time[-1])
#ax1.set_xlim(datetime.datetime(2021,1,1), time[-1])
ax1.set_ylabel('temperature (°C)')
ax1.grid()
ax1.legend(loc=(0.25, 0.65))
ax1.set_title('sea surface temperature')
cax1.axis('off')

ensmean = np.mean(temps, axis=0)
pc = ax2.contourf(mpltime_2d, z, ensmean, 10, cmap=cmap)
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_ylabel('depth (m)')
ax2.set_title('temperature (with DA)');
ax2.grid()
ax2.xaxis.axis_date()
ax2.set_ylim(z.max(), z.min())

temp_diff = np.mean(temps, axis=0) - temp[-enstime.size:, :]
contours = np.linspace(-max_temp_diff, max_temp_diff, 21)
pc = ax3.contourf(mpltime_2d, z, temp_diff, cmap='RdBu_r', levels=contours, extend='both')
cb = fig.colorbar(pc, cax=cax3)
cb.set_label('temperature difference (°C)')
ax3.set_ylabel('depth (m)')
ax3.set_title('temperature difference (DA - no DA)');
ax3.grid()
ax3.xaxis.axis_date()
ax3.set_ylim(z.max(), z.min())

fig.tight_layout()

fig.savefig('ensemble_sst.png', dpi=150)

# Physics + biogeochemistry data assimilation

## Create the ensemble

In [ ]:
gotm = eatpy.models.gotm.YAMLEnsemble("gotm.yaml", N)
fabm = eatpy.models.gotm.YAMLEnsemble("fabm.yaml", N)
with gotm, fabm:
    gotm["surface/u10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["surface/v10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["turbulence/turb_param/k_min"] = 5e-6 * np.random.lognormal(sigma=0.3, size=N)
    gotm["fabm/yaml_file"] = fabm.file_paths
    fabm["instances/phy/parameters/mumax0"] *= np.random.lognormal(sigma=0.3, size=N)
    fabm["instances/dia/parameters/mumax0"] *= np.random.lognormal(sigma=0.3, size=N)

## Run data assimilation experiment

In [ ]:
!mpiexec -n 1 python assimilate_sst_chl.py : -n {N} eat-gotm --separate_gotm_yaml

## Load results

In [ ]:
# Load ensemble results
ncs = [netCDF4.Dataset(os.path.join(setup_dir, 'result_%04i.nc' % (i + 1))) for i in range(N)]
enstime = netCDF4.num2date(ncs[0]['time'], ncs[0]['time'].units)
ensmpltime = dates.date2num(enstime)
z = -ncs[0].variables['z'][:, :, 0, 0]
ens = [nc["total_chlorophyll_calculator_result"][:, :, 0, 0] for nc in ncs]
ens_sf = [v[:, -1] for v in ens]

## Plot

In [ ]:
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(figsize=(10,10), nrows=3, ncols=2, sharex='col', width_ratios=[0.95, 0.05], height_ratios=[0.4, 0.3, 0.3])
low = chl_obs[:,1] - chl_obs[:,2]
high = chl_obs[:,3] - chl_obs[:,1]
#ax1.plot_date(obs[:,0], obs[:,1], '.k', alpha=0.4, label='satellite')
ax1.errorbar(chl_obs[:,0], chl_obs[:,1], yerr=[low, high], ecolor='k', elinewidth=1., fmt='.k', alpha=0.4, zorder=-1, label='satellite')
ax1.plot_date(time, chl_sf, '-', color='C0', label='model, no DA')
ax1.plot_date(enstime, np.mean(ens_sf, axis=0), '-', color='C1', label='model, phys+bgc DA')
ax1.set_xlim(time[0], time[-1])
#ax1.set_xlim(datetime.datetime(2021,1,1), time[-1])
ax1.set_ylabel(f'chlorophyll (mg m-3)')
ax1.grid()
ax1.legend()
ax1.set_title('surface chlorophyll')
ax1.set_ylim(0, 4)
cax1.axis('off')

chl_contours = np.linspace(0., 5., 11)
mpltime_2d = np.broadcast_to(ensmpltime[:, np.newaxis], z.shape)
ens_mean = np.mean(ens, axis=0)
ens_diff = ens_mean - chl[-enstime.size:, :]
pc = ax2.contourf(mpltime_2d, z, chl, chl_contours, cmap=cmap, extend='max')
cb = fig.colorbar(pc, cax=cax2)
cb.set_label('chlorophyll (mg m-3)')
ax2.set_ylabel('height (m)')
ax2.set_title(f'simulated chlorophyll (no DA)');
ax2.grid()
ax2.xaxis.axis_date()
ax2.set_ylim(z.max(), z.min())

#pc = ax3.contourf(mpltime_2d, z, temp_diff, cmap='RdBu_r', levels=np.linspace(-3.5,3.5,21), extend='both')
pc = ax3.contourf(mpltime_2d, z, ens_mean, chl_contours, cmap=cmap, extend='max')
cb = fig.colorbar(pc, cax=cax3)
cb.set_label('chlorophyll (mg m-3)')
ax3.set_ylabel('height (m)')
ax3.set_title(f'simulated chlorophyll (phys+bgc DA)');
ax3.grid()
ax3.xaxis.axis_date()
ax3.set_ylim(z.max(), z.min())

fig.tight_layout()

fig.savefig('ensemble_sst_chl.png', dpi=150)